Below a skeleton class and associated test functions for the `fprop`, `bprop` and `grads_wrt_params` methods of the ConvolutionalLayer class are included.

The test functions assume that in your implementation of `fprop` for the convolutional layer, outputs are calculated only for 'valid' overlaps of the kernel filters with the input - i.e. without any padding.

It is also assumed that if convolutions with non-unit strides are implemented the default behaviour is to take unit-strides, with the test cases only correct for unit strides in both directions.

The three test functions are defined in the cell below. All the functions take as first argument the *class* corresponding to the convolutional layer implementation to be tested (**not** an instance of the class). It is assumed the class being tested has an `__init__` method with at least all of the arguments defined in the skeleton definition above. A boolean second argument to each function can be used to specify if the layer implements a cross-correlation or convolution based operation (see note in [seventh lecture slides](http://www.inf.ed.ac.uk/teaching/courses/mlp/2016/mlp07-cnn.pdf)).

In [83]:
%load_ext autoreload
%autoreload 2
import numpy as np

def test_conv_layer_fprop(layer_class, do_cross_correlation=False):
    """Tests `fprop` method of a convolutional layer.
    
    Checks the outputs of `fprop` method for a fixed input against known
    reference values for the outputs and raises an AssertionError if
    the outputted values are not consistent with the reference values. If
    tests are all passed returns True.
    
    Args:
        layer_class: Convolutional layer implementation following the 
            interface defined in the provided skeleton class.
        do_cross_correlation: Whether the layer implements an operation
            corresponding to cross-correlation (True) i.e kernels are
            not flipped before sliding over inputs, or convolution
            (False) with filters being flipped.

    Raises:
        AssertionError: Raised if output of `layer.fprop` is inconsistent 
            with reference values either in shape or values.
    """
    inputs = np.arange(96).reshape((2, 3, 4, 4))
    kernels = np.arange(-12, 12).reshape((2, 3, 2, 2))
    if do_cross_correlation:
        kernels = kernels[:, :, ::-1, ::-1]
    biases = np.arange(2)
    true_output = np.array(
        [[[[ -958., -1036., -1114.],
           [-1270., -1348., -1426.],
           [-1582., -1660., -1738.]],
          [[ 1707.,  1773.,  1839.],
           [ 1971.,  2037.,  2103.],
           [ 2235.,  2301.,  2367.]]],
         [[[-4702., -4780., -4858.],
           [-5014., -5092., -5170.],
           [-5326., -5404., -5482.]],
          [[ 4875.,  4941.,  5007.],
           [ 5139.,  5205.,  5271.],
           [ 5403.,  5469.,  5535.]]]]
    )
    
    layer = layer_class(
        num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3]
    )
    layer.params = [kernels, biases]
    layer_output = layer.fprop(inputs)
    
    assert layer_output.shape == true_output.shape, (
        'Layer fprop gives incorrect shaped output. '
        'Correct shape is \n\n{0}\n\n but returned shape is \n\n{1}.'
        .format(true_output.shape, layer_output.shape)
    )
    assert np.allclose(layer_output, true_output), (
        'Layer fprop does not give correct output. '
        'Correct output is \n\n{0}\n\n but returned output is \n\n{1}\n\n difference is \n\n{2}.'
        .format(true_output, layer_output, true_output-layer_output)
    )
    return True

def test_conv_layer_bprop(layer_class, do_cross_correlation=False):
    """Tests `bprop` method of a convolutional layer.
    
    Checks the outputs of `bprop` method for a fixed input against known
    reference values for the gradients with respect to inputs and raises 
    an AssertionError if the returned values are not consistent with the
    reference values. If tests are all passed returns True.
    
    Args:
        layer_class: Convolutional layer implementation following the 
            interface defined in the provided skeleton class.
        do_cross_correlation: Whether the layer implements an operation
            corresponding to cross-correlation (True) i.e kernels are
            not flipped before sliding over inputs, or convolution
            (False) with filters being flipped.

    Raises:
        AssertionError: Raised if output of `layer.bprop` is inconsistent 
            with reference values either in shape or values.
    """
    inputs = np.arange(96).reshape((2, 3, 4, 4))
    kernels = np.arange(-12, 12).reshape((2, 3, 2, 2))
    if do_cross_correlation:
        kernels = kernels[:, :, ::-1, ::-1]
    biases = np.arange(2)
    grads_wrt_outputs = np.arange(-20, 16).reshape((2, 2, 3, 3))
    outputs = np.array(
        [[[[ -958., -1036., -1114.],
           [-1270., -1348., -1426.],
           [-1582., -1660., -1738.]],
          [[ 1707.,  1773.,  1839.],
           [ 1971.,  2037.,  2103.],
           [ 2235.,  2301.,  2367.]]],
         [[[-4702., -4780., -4858.],
           [-5014., -5092., -5170.],
           [-5326., -5404., -5482.]],
          [[ 4875.,  4941.,  5007.],
           [ 5139.,  5205.,  5271.],
           [ 5403.,  5469.,  5535.]]]]
    )
    true_grads_wrt_inputs = np.array(
      [[[[ 147.,  319.,  305.,  162.],
         [ 338.,  716.,  680.,  354.],
         [ 290.,  608.,  572.,  294.],
         [ 149.,  307.,  285.,  144.]],
        [[  23.,   79.,   81.,   54.],
         [ 114.,  284.,  280.,  162.],
         [ 114.,  272.,  268.,  150.],
         [  73.,  163.,  157.,   84.]],
        [[-101., -161., -143.,  -54.],
         [-110., -148., -120.,  -30.],
         [ -62.,  -64.,  -36.,    6.],
         [  -3.,   19.,   29.,   24.]]],
       [[[  39.,   67.,   53.,   18.],
         [  50.,   68.,   32.,   -6.],
         [   2.,  -40.,  -76.,  -66.],
         [ -31.,  -89., -111.,  -72.]],
        [[  59.,  115.,  117.,   54.],
         [ 114.,  212.,  208.,   90.],
         [ 114.,  200.,  196.,   78.],
         [  37.,   55.,   49.,   12.]],
        [[  79.,  163.,  181.,   90.],
         [ 178.,  356.,  384.,  186.],
         [ 226.,  440.,  468.,  222.],
         [ 105.,  199.,  209.,   96.]]]])
    layer = layer_class(
        num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3]
    )
    layer.params = [kernels, biases]
    layer_grads_wrt_inputs = layer.bprop(inputs, outputs, grads_wrt_outputs)
    assert layer_grads_wrt_inputs.shape == true_grads_wrt_inputs.shape, (
        'Layer bprop returns incorrect shaped array. '
        'Correct shape is \n\n{0}\n\n but returned shape is \n\n{1}.'
        .format(true_grads_wrt_inputs.shape, layer_grads_wrt_inputs.shape)
    )
    assert np.allclose(layer_grads_wrt_inputs, true_grads_wrt_inputs), (
        'Layer bprop does not return correct values. '
        'Correct output is \n\n{0}\n\n but returned output is \n\n{1}\n\n difference is \n\n{2}'
        .format(true_grads_wrt_inputs, layer_grads_wrt_inputs, layer_grads_wrt_inputs-true_grads_wrt_inputs)
    )
    return True

def test_conv_layer_grad_wrt_params(
        layer_class, do_cross_correlation=False):
    """Tests `grad_wrt_params` method of a convolutional layer.
    
    Checks the outputs of `grad_wrt_params` method for fixed inputs 
    against known reference values for the gradients with respect to 
    kernels and biases, and raises an AssertionError if the returned
    values are not consistent with the reference values. If tests
    are all passed returns True.
    
    Args:
        layer_class: Convolutional layer implementation following the 
            interface defined in the provided skeleton class.
        do_cross_correlation: Whether the layer implements an operation
            corresponding to cross-correlation (True) i.e kernels are
            not flipped before sliding over inputs, or convolution
            (False) with filters being flipped.

    Raises:
        AssertionError: Raised if output of `layer.bprop` is inconsistent 
            with reference values either in shape or values.
    """
    inputs = np.arange(96).reshape((2, 3, 4, 4))
    kernels = np.arange(-12, 12).reshape((2, 3, 2, 2))
    biases = np.arange(2)
    grads_wrt_outputs = np.arange(-20, 16).reshape((2, 2, 3, 3))
    true_kernel_grads = np.array(
        [[[[ -240.,  -114.],
         [  264.,   390.]],
        [[-2256., -2130.],
         [-1752., -1626.]],
        [[-4272., -4146.],
         [-3768., -3642.]]],
       [[[ 5268.,  5232.],
         [ 5124.,  5088.]],
        [[ 5844.,  5808.],
         [ 5700.,  5664.]],
        [[ 6420.,  6384.],
         [ 6276.,  6240.]]]])
    if do_cross_correlation:
        kernels = kernels[:, :, ::-1, ::-1]
        true_kernel_grads = true_kernel_grads[:, :, ::-1, ::-1]
    true_bias_grads = np.array([-126.,   36.])
    layer = layer_class(
        num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3]
    )
    layer.params = [kernels, biases]
    ##########################################################
    # call fprop and bprop first to generate imtermidiate data
    #layer.fprop(inputs)
    #layer.bprop(inputs,)
    ##########################################################
    layer_kernel_grads, layer_bias_grads = (
        layer.grads_wrt_params(inputs, grads_wrt_outputs))
    assert layer_kernel_grads.shape == true_kernel_grads.shape, (
        'grads_wrt_params gives incorrect shaped kernel gradients output. '
        'Correct shape is \n\n{0}\n\n but returned shape is \n\n{1}.'
        .format(true_kernel_grads.shape, layer_kernel_grads.shape)
    )
    assert np.allclose(layer_kernel_grads, true_kernel_grads), (
        'grads_wrt_params does not give correct kernel gradients output. '
        'Correct output is \n\n{0}\n\n but returned output is \n\n{1}.'
        .format(true_kernel_grads, layer_kernel_grads)
    )
    assert layer_bias_grads.shape == true_bias_grads.shape, (
        'grads_wrt_params gives incorrect shaped bias gradients output. '
        'Correct shape is \n\n{0}\n\n but returned shape is \n\n{1}.'
        .format(true_bias_grads.shape, layer_bias_grads.shape)
    )
    assert np.allclose(layer_bias_grads, true_bias_grads), (
        'grads_wrt_params does not give correct bias gradients output. '
        'Correct output is \n\n{0}\n\n but returned output is \n\n{1}.'
        .format(true_bias_grads, layer_bias_grads)
    )
    return True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


An example of using the test functions if given in the cell below. This assumes you implement a convolution (rather than cross-correlation) operation. If the implementation is correct 

In [85]:
from mlp.layers import ConvolutionalLayer
fprop_correct = test_conv_layer_fprop(ConvolutionalLayer, True)
bprop_correct = test_conv_layer_bprop(ConvolutionalLayer, True)
grads_wrt_param_correct = test_conv_layer_grad_wrt_params(ConvolutionalLayer, True)
if fprop_correct and grads_wrt_param_correct and bprop_correct:
    print('All tests passed.')

All tests passed.


In [95]:
%load_ext autoreload
%autoreload 2
%alias_magic t timeit
from scipy.signal import convolve2d
import numpy as np
from mlp.layers import MaxPoolingLayer, ReshapeLayer,ConvolutionalLayer
#inputs = np.arange(96).reshape((2, 3, 4, 4))
def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))



grads_wrt_outputs = np.arange(-20, 4).reshape((2, 3, 2, 2))
layer = MaxPoolingLayer()
def test():
    x_shape = (2, 3, 4, 4)
    inputs = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
    out =  layer.fprop(inputs)
    #return layer.bprop_fast(inputs, outputs, grads_wrt_outputs)
    correct_out = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])
    print('Testing max_pool_forward_naive function:')
    print(out)
    #print( 'difference: ', rel_error(out, correct_out))

def test_naive():

    return layer.fprop_naive(inputs)    
    #return layer.bprop(inputs, outputs, grads_wrt_outputs)

def test_bprop():
    inputs = np.random.randn(3, 2, 8, 8)
    dx_num = eval_numerical_gradient_array(lambda x: max_pool_forward_naive(x, pool_param)[0], x, dout)

    dout = np.random.randn(3, 2, 4, 4)
    outputs = layer.fprop(inputs)
    out = layer.bprop(inputs, outputs, dout)

def c():
    inputs = np.random.randn(100, 1, 28, 28)
    kernels = np.random.randn(5, 1, 2, 2)
    Clayer = ConvolutionalLayer(num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3])
    output = Clayer.fprop(inputs)
    da = np.ones_like(output)
    din = Clayer.bprop(inputs,output, da)
    
    return Clayer.grads_wrt_params(inputs,da)
def c_naive():
    inputs = np.random.randn(100, 1, 28, 28)
    kernels = np.random.randn(5, 1, 2, 2)
    Clayer = ConvolutionalLayer(num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3])
    output = Clayer.fprop_naive(inputs)
    da = np.ones_like(output)
    din = Clayer.bprop_naive(inputs,output, da)
    
    return Clayer.grads_wrt_params_naive(inputs,da)

def c_f():
    inputs = np.random.randn(2, 1, 4, 4)
    kernels = np.random.randn(1, 1, 2, 2)
    Clayer = ConvolutionalLayer(num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3])
    output = Clayer.con2(inputs)
    
    
    return Clayer.bprop(inputs,output1, np.ones_like(output1))

def c_con2d():
    inputs = np.ones((2, 1, 4, 4))*10
    kernels = np.ones((1, 1, 1, 1))
    
    output2 = []
    for i in range(inputs.shape[0]):
        
        output2.append(convolve2d(inputs[i,0,...], kernels[0,0,...],mode='valid'))
    return output2

def im2col():
    inputs = np.random.randn(100, 1, 28, 28)
    kernels = np.random.randn(5, 1, 2, 2)
    Clayer = ConvolutionalLayer(num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3])
    output = Clayer.fprop_im2col(inputs)
    da = np.ones_like(output)
    Clayer.bprop(inputs,output, da)
    return Clayer.grads_wrt_params_naive(inputs,da)
    #return output
    
def stride():
    inputs = np.random.randn(100, 1, 28, 28)
    kernels = np.random.randn(5, 1, 2, 2)
    Clayer = ConvolutionalLayer(num_input_channels=kernels.shape[1], 
        num_output_channels=kernels.shape[0], 
        input_dim_1=inputs.shape[2], 
        input_dim_2=inputs.shape[3],
        kernel_dim_1=kernels.shape[2],
        kernel_dim_2=kernels.shape[3])
    output = Clayer.fprop(inputs)
    da = np.ones_like(output)
    Clayer.bprop(inputs,output, da)
    return Clayer.grads_wrt_params_naive(inputs,da)
    #return output
    
    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [96]:
%t im2col()

2.15 s ± 43.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [97]:
%t stride()

2.08 s ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [255]:
%t c()

13.1 s ± 1.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [273]:
%t c_naive()

34.9 s ± 7.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [284]:
a= c_f()
print(a.shape)
#a = numpy.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
#np.savetxt("foo.csv", a, delimiter=",")
np.savetxt("foo.csv", a[0,0,...], delimiter=",")
with open('test.csv','wb') as f:
    np.savetxt(f,a[0,0,...],)

(2, 1, 4, 4)


In [ ]:
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
pool_param = {'pool_width': 2, 'pool_height': 2, 'stride': 2}

out, _ = max_pool_forward_naive(x, pool_param)

correct_out = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])

# Compare your output with ours. Difference should be around 1e-8.
print 'Testing max_pool_forward_naive function:'
print 'difference: ', rel_error(out, correct_out)

In [25]:
x_shape = (2, 3, 4, 4)
inputs = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
out = layer.fprop(inputs)


ReshapeLayer((3,4,4)).fprop(out).shape




(2, 3, 4, 4)

In [248]:
%t test_naive()

1.1 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [168]:
outputs.shape
#inputs.shape

(2, 3, 2, 2)

In [178]:
np.amax(inputs[:,0,:,:],axis=(-2,-1))

array([15, 63])

In [241]:
x=inputs[:,0,:,:] 
print(x.shape)
mask = x == np.amax(x,axis=(-2,-1))
mask #* grads_wrt_outputs[:, 0, 0,0][..., None,None]
#(x[:,...]== np.amax(x[:,...],axis=(-2,-1))[...,None])
#np.ma.masked_where(x== np.amax(x,axis=(-2,-1))[...,None],x)
#np.amax(x,axis=(-2,-1))[...,None].shape


(2, 4, 4)


D:\Anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


False

In [292]:
def get_im2col_indices(x_shape, field_height, field_width, padding=1, stride=1):
    # First figure out what the size of the output should be
    N, C, H, W = x_shape
    assert (H + 2 * padding - field_height) % stride == 0
    assert (W + 2 * padding - field_height) % stride == 0
    out_height = (H + 2 * padding - field_height) // stride + 1
    out_width = (W + 2 * padding - field_width) // stride + 1

    i0 = np.repeat(np.arange(field_height), field_width)
    i0 = np.tile(i0, C)
    i1 = stride * np.repeat(np.arange(out_height), out_width)
    j0 = np.tile(np.arange(field_width), field_height * C)
    j1 = stride * np.tile(np.arange(out_width), out_height)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)

    k = np.repeat(np.arange(C), field_height * field_width).reshape(-1, 1)

    return (k, i, j)

In [297]:
print(inputs.shape)
k,i,j = get_im2col_indices(inputs.shape, 2,2,0)


(2, 3, 4, 4)


array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3],
       [0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3],
       [0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3],
       [0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3],
       [0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3],
       [0, 1, 2, 0, 1, 2, 0, 1, 2],
       [1, 2, 3, 1, 2, 3, 1, 2, 3]])

In [300]:
cols =inputs[:,k,i,j]
cols.transpose(1,2,0).reshape(2*2*3,-1)

array([[ 0, 48,  1, 49,  2, 50,  4, 52,  5, 53,  6, 54,  8, 56,  9, 57, 10,
        58],
       [ 1, 49,  2, 50,  3, 51,  5, 53,  6, 54,  7, 55,  9, 57, 10, 58, 11,
        59],
       [ 4, 52,  5, 53,  6, 54,  8, 56,  9, 57, 10, 58, 12, 60, 13, 61, 14,
        62],
       [ 5, 53,  6, 54,  7, 55,  9, 57, 10, 58, 11, 59, 13, 61, 14, 62, 15,
        63],
       [16, 64, 17, 65, 18, 66, 20, 68, 21, 69, 22, 70, 24, 72, 25, 73, 26,
        74],
       [17, 65, 18, 66, 19, 67, 21, 69, 22, 70, 23, 71, 25, 73, 26, 74, 27,
        75],
       [20, 68, 21, 69, 22, 70, 24, 72, 25, 73, 26, 74, 28, 76, 29, 77, 30,
        78],
       [21, 69, 22, 70, 23, 71, 25, 73, 26, 74, 27, 75, 29, 77, 30, 78, 31,
        79],
       [32, 80, 33, 81, 34, 82, 36, 84, 37, 85, 38, 86, 40, 88, 41, 89, 42,
        90],
       [33, 81, 34, 82, 35, 83, 37, 85, 38, 86, 39, 87, 41, 89, 42, 90, 43,
        91],
       [36, 84, 37, 85, 38, 86, 40, 88, 41, 89, 42, 90, 44, 92, 45, 93, 46,
        94],
       [37, 85, 38, 8

In [299]:
inputs%MCEPASTEBIN%%MCEPASTEBIN%

array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39],
         [40, 41, 42, 43],
         [44, 45, 46, 47]]],


       [[[48, 49, 50, 51],
         [52, 53, 54, 55],
         [56, 57, 58, 59],
         [60, 61, 62, 63]],

        [[64, 65, 66, 67],
         [68, 69, 70, 71],
         [72, 73, 74, 75],
         [76, 77, 78, 79]],

        [[80, 81, 82, 83],
         [84, 85, 86, 87],
         [88, 89, 90, 91],
         [92, 93, 94, 95]]]])

In [6]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [42]:
s = (2,5,6)
np.array(s)

array([2, 5, 6])